In [1]:
from sklearn.datasets import fetch_openml
import numpy as np
import gc

In [2]:
## Computing the full singular value decomposition for the F-MNIST dataset
X, y = fetch_openml(data_id=40996, return_X_y=True)

#Remove mean before applying PCA
X_mean0 = np.array(X - np.mean(X, axis = 0))

_, E, _ = np.linalg.svd(X_mean0, full_matrices=False)
spectral_norm = E[0]
A = X_mean0/spectral_norm

In [3]:
def __mu(p, matrix):
    def s(p, A):
        if p == 0:
            result = np.max([np.count_nonzero(A[i]) for i in range(len(A))])
        else:
            norms = np.sum(np.power(np.abs(A), p), axis=1)
            result = max(norms)
            del norms
        gc.collect()
        return result

    s1 = s(2 * p, matrix)
    s2 = s(2 * (1 - p), matrix.T)
    mu = np.sqrt(s1 * s2)

    gc.collect()
    return mu

def linear_search(matrix, start=0.0, end=1.0, step=0.05):
    domain = [i for i in np.arange(start, end, step)] + [end]
    values = [__mu(i, matrix) for i in domain]
    best_p = domain[values.index(min(values))]
    return best_p, min(values)

def best_mu(matrix, start=0.0, end=1.0, step=0.05):
    p, val = linear_search(matrix, start=start, end=end, step=step)
    val_list = [val, np.linalg.norm(matrix)]
    index = val_list.index(min(val_list))
    if index == 0:
        best_norm = f"p={p}"
    elif index == 1:
        best_norm = "Frobenius"        

    return best_norm, val_list[index]

In [4]:
res_nonnorm = best_mu(X_mean0, step=0.05)
res_norm = best_mu(A, step=0.05)

print(f"non-norm best:\t{res_nonnorm[0]}")
print(f"non-norm mu:\t{res_nonnorm[1]}")
print(f"norm best:\t{res_norm[0]}")
print(f"norm mu:\t{res_norm[1]}")

non-norm best:	Frobenius
non-norm mu:	557058.9124795411
norm best:	Frobenius
norm mu:	1.855145802991925
